In [1]:
from urllib.request import urlopen
import pandas as pd
import urllib
import json 
import os

In [ ]:
## Read stations from selected stations csv
stations = pd.read_csv("ches_daily_selected_stations.csv")

In [3]:
## Reformat data from the csv file
stations['ID'] = [x.split(' ')[0] for x in stations['Longterm SID'].values]
stations.set_index('ID', inplace=True)

In [ ]:
## For loop to get station time series from the Cornell API

for stn in stations.index:
    
    yr_start = str(stations.loc[stn, 'POR Start'])
    yr_end = str(stations.loc[stn, 'POR End'])
    duration_val = 1
    input_dict = {"sid":stn,"sdate":yr_start+"-01-01","edate":yr_end+"-09-10","elems":[{"name":"pcpn","interval":"dly","duration":duration_val,"reduce":"sum","maxmissing":"0"}]}    
    params = urllib.parse.urlencode({'params':json.dumps(input_dict)})
    params = params.encode('ascii')
    req = urllib.request.Request('http://data.rcc-acis.org/StnData', params, {'Accept':'application/json'})
    with urllib.request.urlopen(req) as response:
        the_page = response.read()
    data_vals = json.loads(the_page)


    date = [x[0] for x in data_vals['data']]
    prcp_vals = [x[1] for x in data_vals['data']]

    station_df = pd.DataFrame(prcp_vals, date, columns=['prcp'])
    
    if not os.path.isdir("station_data/ts"):
        os.makedirs("station_data/ts")
    station_df.to_csv("station_data/ts/{}.csv".format(stn))